In [1]:
import torch
import torchvision.models as models
from torch.profiler import profile, ProfilerActivity

model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
   model(inputs)



STAGE:2023-01-05 13:24:03 59758:587969 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-05 13:24:04 59758:587969 ActivityProfilerController.cpp:300] Completed Stage: Collection


In [2]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.17%     959.000us        91.07%     508.678ms      25.434ms            20  
                aten::convolution         0.14%     771.000us        90.89%     507.719ms      25.386ms            20  
               aten::_convolution         0.36%       2.005ms        90.76%     506.948ms      25.347ms            20  
         aten::mkldnn_convolution        90.30%     504.402ms        90.40%     504.943ms      25.247ms            20  
                 aten::batch_norm         0.05%     289.000us         3.20%      17.876ms     893.800us            20  
     aten::_batch_norm_impl_index       

In [3]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=30))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                     aten::conv2d         0.00%      20.000us        21.30%     118.987ms      29.747ms             4                             [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], []]  
                aten::convolution         0.01%      61.000us        21.30%     118.967ms      29.742ms             4                     [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], [], 

Profile the memory allocation too.

In [5]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof:
   model(inputs)

STAGE:2023-01-05 13:25:05 59758:587969 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-05 13:25:06 59758:587969 ActivityProfilerController.cpp:300] Completed Stage: Collection


In [6]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.08%     378.000us         0.08%     378.000us       1.890us      94.83 Mb      94.83 Mb           200  
    aten::max_pool2d_with_indices         3.00%      14.855ms         3.00%      14.855ms      14.855ms      11.48 Mb      11.48 Mb             1  
                 aten::empty_like         0.01%      62.000us         0.02%     123.000us       6.150us      47.37 Mb     980.00 Kb            20  
                     aten::conv2d         0.55%       2.732ms        93.04%     460.827ms      23.041ms      47.

Export the profiling.

In [7]:
prof.export_chrome_trace("trace.json")


STAGE:2023-01-05 13:25:40 59758:587969 output_json.cpp:417] Completed Stage: Post Processing


Profile multiple steps to get a better understanding.

In [8]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True) as prof:
   for i in range(10):
      model(inputs)
      prof.step()


STAGE:2023-01-05 13:27:10 59758:587969 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-05 13:27:14 59758:587969 ActivityProfilerController.cpp:300] Completed Stage: Collection


In [9]:
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.07%       3.288ms         0.07%       3.288ms       1.644us     947.37 Mb     947.37 Mb          2000  
    aten::max_pool2d_with_indices         2.16%     102.916ms         2.16%     102.916ms      10.292ms     114.84 Mb     114.84 Mb            10  
                 aten::empty_like         0.01%     560.000us         0.02%       1.101ms       5.505us     473.73 Mb      15.31 Mb           200  
               aten::_convolution         0.09%       4.482ms        94.44%        4.509s      22.544ms     473.

## using tensorboard

In [16]:
from torch.profiler import profile, tensorboard_trace_handler
with profile(activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True, on_trace_ready=tensorboard_trace_handler("./log/resnet18")) as prof:
         model(inputs)



STAGE:2023-01-05 13:58:25 59758:587969 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-05 13:58:25 59758:587969 ActivityProfilerController.cpp:300] Completed Stage: Collection
STAGE:2023-01-05 13:58:25 59758:587969 output_json.cpp:417] Completed Stage: Post Processing


In [19]:

!tensorboard --logdir="./log/resnet18/" --port=8892


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

I0105 14:07:17.884896 12940980224 plugin.py:429] Monitor runs begin
I0105 14:07:17.885462 12940980224 plugin.py:444] Find run directory /Users/alaina/Desktop/classes/2023Jan/dtu_mlops/s4_debugging_and_logging/exercise_files/log/resnet18
I0105 14:07:17.886278 12974559232 plugin.py:493] Load run resnet18
I0105 14:07:17.968463 12974559232 loader.py:57] started all processing
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.0 at http://localhost:8892/ (Press CTRL+C to quit)
I0105 14:07:19.681622 12974559232 plugin.py:497] Run resnet18 loaded
I0105 14:07:19.682111 12957769728 plugin.py:467] Add run resnet18
W0105 14:07:35.864212 13008674816 security_validator.py:46] In 3.0, this warning will become an error:
Requires default-src for Content-Se